## Statistical properties of the MRR process

In [6]:
import numpy as np
import scipy
import itertools

import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from stats.mrr import mrr
from stats.common import acvar

from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [7]:
def population_cm_2(c_l, c_r, sigma, rho):
    """
    Population second central moment for m=0
    """
    return sigma**2 + (c_l+c_r)**2 + 2.*(rho-1.)*c_l*c_r


def population_cm_4(c_l, c_r, sigma, rho):
    """
    Population forth central moment for m=0
    """
    return (c_l+c_r)**4 + 4.*(rho-1.)*c_l*c_r*(c_l**2+c_r**2) + 6.*sigma**2*((c_l+c_r)**2 + 2.*(rho-1.)*c_l*c_r) + 3.*sigma**4


def acv_prefactor(c_l, c_r, rho):
    """
    Auto-covariance prefactor for m=0
    """
    return (c_l**2+c_r**2+c_l*c_r*(rho+1./rho))


def acv_population(x, a, b):
    """ Functional form of the auto-covariance function """
    return a * b**x


def equations_set(x, y_rho):

    x_0, x_1, x_2 = x
    y_0, y_1, y_2, rho = y_rho

    return (population_cm_2(x_0, x_1, x_2, rho) - y_0,
            population_cm_4(x_0, x_1, x_2, rho) - y_1,
            acv_prefactor(x_0, x_1, rho) - y_2)

### Autocovariance of the time series of $y_i$

In [4]:
n_points = 2**15
n_lags = 6

m = 0.
theta = 0.5
phi = 0.5
sigma = 1.
rho_list = [0.1, 0.3, 0.5, 0.7]
color_list = ['red', 'blue', 'green', 'black']

p = figure(plot_width = 500, plot_height=300)


for rho in rho_list:
    
    # generate the time series of returns with the MRR price dynamics
    yy_0 = mrr(n_points, m=m, rho=rho, theta=theta, phi=phi, sigma=sigma)
    yy = np.array([i[1] for i in yy_0])

    # sample auto-covariance
    list_lags = list(range(1,n_lags))
    acv_sample = acvar(yy, n_lags)[1:]

    # population auto-covariance
    xx = np.linspace(1, n_lags, 1000)
    c_l = theta + phi
    c_r = -(rho*theta + phi)
    acv_population = (1-m**2)*(c_l**2+c_r**2+c_l*c_r*(rho+1./rho))*np.power(rho,xx)
    
    cc = color_list[rho_list.index(rho)]
    p.scatter(list_lags, acv_sample, legend=str(rho), fill_color='white', line_color=cc, size=4)
    p.line(xx, acv_population, line_color=cc)

show(p)


### Variance of the stationary marginal distribution of $y_i$

In [11]:
n_points = 2**15
n_sample = 2**10
n_lags = 8

rho_true = []
rho_hat = []

theta_true = []
phi_true = []
sigma_true = []
cl_true = []
cr_true = []

var_sample = []
var_population = []

kurtosis_sample = []
kurtosis_population = []

prefactor_sample = []
prefactor_population = []


cc_wrong = 0
for i in range(n_sample):

    # generate random model parameters
    m = 0.
    rho = 0.1 + 0.8*np.random.rand()
    theta = np.random.rand()
    phi = np.random.rand()
    sigma = np.random.rand()

    c_l = theta + phi
    c_r = -(rho*theta + phi)

    # generate the time series of returns with the MRR price dynamics
    yy_0 = mrr(n_points, m=m, rho=rho, theta=theta, phi=phi, sigma=sigma)
    yy = np.array([i[1] for i in yy_0])

    # estimate the sample auto-covariance
    list_lags = list(range(1,n_lags+1))
    acv_sample = acvar(yy, n_lags)[1:]

    cc = 0
    out = None
    
    while out is None and cc<5:
        try:
            
            # try non-linear fit of the auto-correlation function
            pr_start = -2.*np.random.rand()
            rho_start = np.random.rand()
            out = scipy.optimize.curve_fit(acv_population, list_lags, acv_sample, [pr_start, rho_start])
        
        except:
            cc += 1
            if cc == 5:
                cc_wrong += 1
            pass
        
        else:
            
            # append true parameters
            rho_true.append(rho)
            theta_true.append(theta)
            phi_true.append(phi)
            sigma_true.append(sigma)
            cl_true.append(c_l)
            cr_true.append(c_r)

            # append population moments
            prefactor_population.append(acv_prefactor(c_l, c_r, rho))
            var_population.append(population_cm_2(c_l, c_r, sigma, rho))
            kurtosis_population.append(population_cm_4(c_l, c_r, sigma, rho))

            # append estimated prefactor and rho
            prefactor_sample.append(out[0][0])
            var_sample.append(np.var(yy))
            kurtosis_sample.append(scipy.stats.kurtosis(yy, fisher=False, bias=False) * np.var(yy)**2)
            
            # append hat rho
            rho_hat.append(out[0][1])

print((1.*cc_wrong)/n_sample)
print(np.std((np.array(rho_hat)-np.array(rho_true))/np.array(rho_true)))

0.015625
0.304642499282


In [12]:
pw = 400
ph = 300

# plot variance
p1 = figure(plot_width = pw, plot_height=ph)
p1.circle(var_population, var_sample, size =2)

# plot kurtosis
p2 = figure(plot_width = pw, plot_height=ph)
p2.circle(kurtosis_population, kurtosis_sample, size =2)

# plot rho
p3 = figure(plot_width = pw, plot_height=ph, x_range=[0.,1.], y_range=[-0.1,1.1])
p3.circle(rho_true, rho_hat, size =2)
p3.line(np.linspace(0,1,10), np.linspace(0,1,10), line_color='red')

# plot prefactor
p4 = figure(plot_width = pw, plot_height=ph, y_range = [-12, 1.], x_range=[-10,0])
p4.circle(prefactor_population, prefactor_sample, size =2)
p4.line(np.linspace(-16,0,10), np.linspace(-16,0,10), line_color='red')

show(column(row(p1,p2),row(p3,p4)))

In [13]:

sigma_hat = []
cl_hat = []
cr_hat = []

for y_rho in zip(var_sample, kurtosis_sample, prefactor_sample, rho_hat):

    try:        
        sigma_h = -1.
        cc = 0
        
        while sigma_h < 0. or cl_h < 0. and cc<10:
            
            # random starting point for the serach of c_l, c_r and sigma
            x_start = [np.random.rand() + 1., 
                       -1.*np.random.rand() -0.5, 
                       np.random.rand()]

            # find numerically the solution of the equations
            cl_h, cr_h, sigma_h = tuple(scipy.optimize.fsolve(equations_set, x0=x_start, args=list(y_rho)))
            
            cc += 1

    except:
        cl_h = -100.
        cr_h = -100.
        sigma_h = -100.
        
    finally:
        sigma_hat.append(sigma_h)
        cl_hat.append(cl_h)
        cr_hat.append(cr_h)

p1_1 = figure(plot_width = pw, plot_height=ph, y_range=[-0.5,2.5])
p1_1.circle(cl_true, cl_hat, size =2)
tmp = np.linspace(min(cl_true), max(cl_true), 100)
p1_1.line(tmp, tmp, color='red')

p1_2 = figure(plot_width = pw, plot_height=ph, y_range=[-2.,0.5])
p1_2.circle(cr_true, cr_hat, size =2)
tmp = np.linspace(min(cr_true), max(cr_true), 100)
p1_2.line(tmp, tmp, color='red')

p1_3 = figure(plot_width = pw, plot_height=ph, y_range=[-0.1,1.1])
p1_3.circle(rho_true, rho_hat, size =2)
tmp = np.linspace(min(rho_true), max(rho_true), 100)
p1_3.line(tmp, tmp, color='red')

p1_4 = figure(plot_width = pw, plot_height=ph, y_range=[0.,1.2])
p1_4.circle(sigma_true, sigma_hat, size =2)
tmp = np.linspace(min(sigma_true), max(sigma_true), 100)
p1_4.line(tmp, tmp, color='red')


show(column(row(p1_1,p1_2), row(p1_3,p1_4)))

/Users/eliazarinelli/anaconda/lib/python3.5/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
/Users/eliazarinelli/anaconda/lib/python3.5/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/eliazarinelli/anaconda/lib/python3.5/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The number of calls to function has reached maxfev = 800.
  warnings.warn(msg, RuntimeWarning)
